In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import transformers

In [4]:
from transformers import BertModel,BertTokenizer
# create a BERT model
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# get the list of layers
layers = list(model.encoder.layer)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


****
## Checking the dataset

In [5]:
import pandas as pd

cola_dataset=pd.read_csv('/Users/yashsurange/adapter_bert/cola_dataset/raw/in_domain_train.tsv',sep='\t')
cola_dataset.columns=['source','acceptibility','originality','sentence']

In [6]:
cola_dataset.head()

,source,acceptibility,originality,sentence
0,gj04,1,NaN,One more pseudo generalization and I'm giving up.
1,gj04,1,NaN,One more pseudo generalization or I'm giving up.
2,gj04,1,NaN,"The more we study verbs, the crazier they get."
3,gj04,1,NaN,Day by day the facts are getting murkier.
4,gj04,1,NaN,I'll fix you a drink.


In [7]:
cola_dataset[['acceptibility','sentence']].head()

,acceptibility,sentence
0,1,One more pseudo generalization and I'm giving up.
1,1,One more pseudo generalization or I'm giving up.
2,1,"The more we study verbs, the crazier they get."
3,1,Day by day the facts are getting murkier.
4,1,I'll fix you a drink.


In [8]:
cola_dataset.iloc[1]

source                                                       gj04
acceptibility                                                   1
originality                                                   NaN
sentence         One more pseudo generalization or I'm giving up.
Name: 1, dtype: object

****
## Checking tokenisation done in dataset.py

In [9]:
path='/Users/yashsurange/adapter_bert/cola_dataset/raw/in_domain_train.tsv'

In [21]:
from dataset import dataset_adapter

train_dataset_tokenised=dataset_adapter(path,tokenizer)

In [11]:
train_dataset_tokenised.data.head()

,acceptibility,sentence
0,1,"Our friends won't buy this analysis, let alone..."
1,1,One more pseudo generalization and I'm giving up.
2,1,One more pseudo generalization or I'm giving up.
3,1,"The more we study verbs, the crazier they get."
4,1,Day by day the facts are getting murkier.


In [20]:
# When maxlength was kept at 15
train_dataset_tokenised.__getitem__(0)['sentence']['input_ids']

tensor([[ 101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106, 1010, 2292, 2894,
         1996, 2279,  102]])

In [22]:
# When max length is kept at 10
train_dataset_tokenised.__getitem__(0)['sentence']['input_ids']

tensor([[ 101, 2256, 2814, 2180, 1005, 1056, 4965, 2023, 4106,  102]])

In [14]:
# When max length is kept at 20. We can see padding i.e padding with zeros 
train_dataset_tokenised.__getitem__(0)['sentence']['input_ids']

tensor([[  101,  2256,  2814,  2180,  1005,  1056,  4965,  2023,  4106,  1010,
          2292,  2894,  1996,  2279,  2028,  2057, 16599,  1012,   102,     0]])

In [15]:
# From token ids to tokens
train_dataset_tokenised.tokenizer.convert_ids_to_tokens(train_dataset_tokenised.__getitem__(3)['sentence']['input_ids'].reshape([-1]))

['[CLS]',
 'the',
 'more',
 'we',
 'study',
 'verbs',
 ',',
 'the',
 'cr',
 '##azi',
 '##er',
 'they',
 'get',
 '.',
 '[SEP]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]',
 '[PAD]']

****


### Why near identidy intialisation is required in adapters ?

In [83]:
mat_after_projection=torch.ones(1,25,dtype=torch.int64)

In [84]:
reshaped_down_projected=mat_after_projection.view([5,5])

In [86]:
# Our linear downward projection

down_prjection=torch.eye([5,3],dtype=torch.int64)

TypeError: eye() received an invalid combination of arguments - got (list, dtype=torch.dtype), but expected one of:
 * (int n, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (int n, int m, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [60]:
# Initialized to identity 
down_prjection

tensor([[1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1],
        [1, 1, 1]])

In [61]:
projection_before_bottlenect=reshaped_down_projected@down_prjection

In [63]:
reshaped_down_projected

tensor([[ 1,  2,  3,  4,  5],
        [ 6,  7,  8,  9, 10],
        [11, 12, 13, 14, 15],
        [16, 17, 18, 19, 20],
        [21, 22, 23, 24, 25]])

In [62]:
projection_before_bottlenect

tensor([[ 15,  15,  15],
        [ 40,  40,  40],
        [ 65,  65,  65],
        [ 90,  90,  90],
        [115, 115, 115]])

In [72]:
after_activation=torch.relu(projection_before_bottlenect)

In [71]:
upward_prjection=torch.ones([3,5],dtype=torch.int64)

In [73]:
final_projection=after_activation@upward_prjection

In [74]:
final_projection

tensor([[ 45,  45,  45,  45,  45],
        [120, 120, 120, 120, 120],
        [195, 195, 195, 195, 195],
        [270, 270, 270, 270, 270],
        [345, 345, 345, 345, 345]])